# Mechanisms of Action (MoA) Prediction

https://www.kaggle.com/c/lish-moa

In [48]:
import numpy as np
import pandas as pd

from sklearn.multioutput import MultiOutputClassifier

from xgboost import XGBClassifier

## Data

In [49]:
#DATA_DIR = '/kaggle/input/lish-moa/'
DATA_DIR = 'lish-moa/'

train_features = pd.read_csv(DATA_DIR + 'train_features.csv')
train_targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test_features = pd.read_csv(DATA_DIR + 'test_features.csv')

test_targets = pd.DataFrame(0, index= range(len(test_features)), columns=train_targets.columns)
test_targets['sig_id'] = test_features['sig_id']

## Preprocess

In [51]:
# Màscara dels registres amb cp_type = ctl_vehicle
ctl_mask = train_features.cp_type=='ctl_vehicle'

# Binarització de cp_dose 
train_features['is_D1'] = train_features['cp_dose'] == 'D1'

# Eliminem columnes
train_features.drop(columns=['sig_id','cp_type','cp_dose'], inplace=True)
train_targets.drop(columns=['sig_id'], inplace=True)

# Dades d'entrenament
X = train_features[~ctl_mask].to_numpy()
Y = train_targets[~ctl_mask].to_numpy()

## Model

In [ ]:
# Paràmetres

ESTIMATORS = 20
MAX_DEPTH = 8

In [ ]:
# Preprocess

test_ctl_mask = test_features.cp_type=='ctl_vehicle'
test_features['is_D1'] = test_features['cp_dose'] == 'D1'
test_features.drop(columns=['sig_id','cp_type','cp_dose'], inplace=True)

test = test_features[~test_ctl_mask].to_numpy()

In [ ]:
# Train

clf = MultiOutputClassifier(
    XGBClassifier(n_estimators= ESTIMATORS, max_depth=MAX_DEPTH, objective='binary:logistic'))

clf.fit(X, Y)

In [ ]:
# Predict

pred = clf.predict_proba(test)
test_targets.loc[~test_ctl_mask,'5-alpha_reductase_inhibitor':] = np.array(pred)[:,:,1].T
test_targets.to_csv('submission.csv', index = False)